In [3]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


Obtain API key from https://opendata.aemet.es/centrodedescargas/

Create .env file in root directory and add:
```AEMET_API_KEY=your_api key```

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
AEMET_API_KEY = os.getenv('AEMET_API_KEY')

print(AEMET_API_KEY[:5]) # Ensure this correctly prints the first 5 characters of your API key

In [32]:
import requests
import pandas as pd

In [17]:
headers = {'api_key': AEMET_API_KEY}

In [9]:
base_url = 'https://opendata.aemet.es/opendata/api'

In [11]:
endpoints = {
    'maestro': {
        'municipio': '/maestro/municipio/{municipio}',
        'municipios': '/maestro/municipios'
    }
}

In [12]:
def replace_params(url, **kwargs):
    for key, value in kwargs.items():
        placeholder = f"{{{key}}}"
        url = url.replace(placeholder, str(value))
    return url

In [18]:
def get_response(url) -> tuple[dict, int]:
    
    try:
        response = requests.get(url=url, headers=headers)
        response.raise_for_status()
        
        return response.json(), response.status_code
        
    except requests.exceptions.RequestException as req_err:
        print(f"Request failed: {req_err}")
        return {"error": str(req_err)}, 500

    except Exception as e:
        print('Error occurred:', e)
        return {"error": str(e)}, 500

In [19]:
municipios = get_response(f"{base_url}/{endpoints['maestro']['municipios']}")

In [20]:
municipios

({'descripcion': 'exito',
  'estado': 200,
  'datos': 'https://opendata.aemet.es/opendata/sh/7b26990d',
  'metadatos': 'https://opendata.aemet.es/opendata/sh/4c38eb58'},
 200)

In [22]:
municipios_datos = get_response(municipios[0]['datos'])

In [26]:
municipios_datos[0][0]

{'latitud': '40º32\'54.450744"',
 'id_old': '44004',
 'url': 'ababuj-id44001',
 'latitud_dec': '40.54845854',
 'altitud': '1372',
 'capital': 'Ababuj',
 'num_hab': '65',
 'zona_comarcal': '624401',
 'destacada': '0',
 'nombre': 'Ababuj',
 'longitud_dec': '-0.80780117',
 'id': 'id44001',
 'longitud': '-0º48\'28.084212"'}

In [27]:
municipios_nombres = [dato['nombre'] for dato in municipios_datos[0]]

In [30]:
municipios_nombres[:5]

['Ababuj', 'Abades', 'Abadiño', 'Abadía', 'Abadín']

In [35]:
municipios_df = pd.DataFrame(municipios_datos[0])

In [36]:
municipios_df.head()

,latitud,id_old,url,latitud_dec,altitud,capital,num_hab,zona_comarcal,destacada,nombre,longitud_dec,id,longitud
0,"40º32'54.450744""",44004,ababuj-id44001,40.54845854,1372,Ababuj,65,624401,0,Ababuj,-0.80780117,id44001,"-0º48'28.084212"""
1,"40º54'58.824504""",40004,abades-id40001,40.91634014,971,Abades,873,674001,0,Abades,-4.26787389,id40001,"-4º16'4.346004"""
2,"43º8'51.525564""",48010,abadino-abadino-zelaieta-id48001,43.14764599,144,Abadiño-Zelaieta,7504,754802,0,Abadiño,-2.60687319,id48001,"-2º36'24.743484"""
3,"40º15'34.315272""",10004,abadia-id10001,40.25953202,451,Abadía,324,701001,0,Abadía,-5.97785806,id10001,"-5º58'40.289016"""
4,"43º21'46.874736""",27010,abadin-abadin-o-provecende-id27001,43.36302076,515,Abadín o Provecende,2646,712702,0,Abadín,-7.47214495,id27001,"-7º28'19.72182"""


In [47]:
municipios_df.to_csv('../datasets/municipios_raw.csv', index=False)